In [3]:
import backend
import json
import generate_page as page
initial_prompt = """
Generate YC’s Co-founder Matching website. It allows entrepreneurs to create a profile, and find the contact information of other entrepreneurs.
It should feature the following pages:
1.  A home landing page advertising the Co-founder matching site, which allows the user to start creating an account using the built-in clerk authentication buttons, and shows marketing items advertising how many prestigious Stanford/MIT cofounders are on here. 
2. Next, after the user creates an account by entering their email, they are taken to a page where they create their own profile. They fill out their profile information, which includes the following: their name, a one liner on their background, a link to a linkedin profile (which is hidden from others until matched), and a checkbox on if they are a technical cofounder or not.
3. After their profile is created, they then can view other profiles and interact with them on the match page. Here, they scroll through a list of other user profiles who do not have a match yet. If they are interested, they can press a match button that sends them a match request as well as a short message.
4. Users can also view their match requests, and either approve or reject them. If a request is rejected, it is removed from the matches. If they approve, the request gets moved to a match section, where they can add the other person on linkedin using their linkedin link.
5. Users can also edit their profile as well. If they found a cofounder, they can mark their profile as matched, and they will be removed from the search.

Schema:
User: email, name, linkedin, description, technical / non technical, available / not available
Match: User from, User to, Accepted / rejected, request message, accept message

"""
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'
test_path = 'test/app'


In [2]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

Here is the schema for the YC Co-founder Matching website:

<jsonSchema>
{
  "description": "The YC Co-founder Matching website allows entrepreneurs to create a profile and find contact information of other entrepreneurs to potentially match with as co-founders. Users can create a profile, view and match with other profiles, manage match requests, and edit their profile.",
  "features_description": "1. Home landing page advertising the site with Clerk authentication to create account\n2. Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status\n3. Match page to view other available user profiles and send match requests with message\n4. View and manage incoming match requests, approve to connect on LinkedIn or reject\n5. Edit profile page and mark profile as matched once co-founder is found",
  "schema_tables": [
    {
      "name": "users",
      "docs": "Stores user profile information",
      "fields": [
        {
          "name": "email",

In [3]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

Here is the generated JSON schema for the backend actions based on the provided application structure:

<jsonSchema>
{
  "actions_description": "The backend actions will handle creating and updating user profiles, retrieving user profiles for the match page, creating and managing match requests between users.",
  "query_actions_required": "Query actions are needed to retrieve a user's own profile, get a list of available user profiles for the match page, and get a user's incoming and outgoing match requests.",
  "query_actions": [
    {
      "name": "getUserProfile",
      "where_used": "Edit Profile page",
      "docs": "Retrieves the profile of the currently logged in user.",
      "requires_auth": true,
      "arguments": [],
      "returns": "The user's profile data",
      "return_type": "User object",
      "workflow_steps": [
        {
          "step": "Get the currently logged in user's ID from Clerk auth"
        },
        {
          "step": "Query the 'users' table for th

In [6]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
actions = backend.read(actions_path)
schema_page = backend.create_schema(schema, schema_page_path)
crud_page = backend.create_crud(schema, crud_path)
faker_data = backend.create_faker_data_code(schema, 'generated/faker.ts')
actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

Here is the code in the requested JSON schema format:

<jsonSchema>
{
  "getUserProfile": "const [identity, user] = await verify(ctx);\nconst userProfile = await getOneUsers(d, user._id);\nreturn userProfile;",
  "getAvailableProfiles": "const [identity, user] = await verify(ctx);\nconst availableProfiles = await getManyUsers(d, (u) => u.isAvailable && u._id != user._id).collect();\nreturn availableProfiles;",
  "getIncomingMatches": "const [identity, user] = await verify(ctx);\nconst incomingMatches = await getManyMatches(d, (m) => m.to == user._id).collect();\nreturn incomingMatches;", 
  "getOutgoingMatches": "const [identity, user] = await verify(ctx);\nconst outgoingMatches = await getManyMatches(d, (m) => m.from == user._id).collect();\nreturn outgoingMatches;",
  "createUserProfile": "const [identity, user] = await verify(ctx);\nconst userId = await createOneUsers(d, {email: identity.email, name: args.name, linkedin: args.linkedin, description: args.description, isTechnical: arg

In [ ]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)

In [7]:
# alternative way to make page structure: simplified version
page.write(page_path, page.generate_component_list(schema_path, actions_path))

{
    "pages": [
        {
            "url": "/",
            "title": "Home",
            "description": "Landing page advertising the site with Clerk authentication to create account"
        },
        {
            "url": "/createProfile",
            "title": "CreateProfile",
            "description": "Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status"  
        },
        {
            "url": "/match",
            "title": "Match",
            "description": "Match page to view other available user profiles and send match requests with message"
        },
        {
            "url": "/matchRequests",
            "title": "MatchRequests", 
            "description": "View and manage incoming match requests, approve to connect on LinkedIn or reject"
        },
        {
            "url": "/editProfile",
            "title": "EditProfile",
            "description": "Edit profile page and mark profile as matched once co-founder 

In [5]:
# generate all the parts
pages = json.loads(backend.read(page_path))["pages"]
# page.generate_pages(test_path, pages, schema_path, actions_path)
page.generate_page(test_path, pages, pages[4], schema_path, actions_path, 0)


You are writing a web app with Next.js, Convex, and Tailwind. Take the following description of the app and a list of pages and components, and edit the boilerplate file. 
The app description is as follows: 
The YC Co-founder Matching website allows entrepreneurs to create a profile and find contact information of other entrepreneurs to potentially match with as co-founders. Users can create a profile, view and match with other profiles, manage match requests, and edit their profile.
Features: 1. Home landing page advertising the site with Clerk authentication to create account
2. Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status
3. Match page to view other available user profiles and send match requests with message
4. View and manage incoming match requests, approve to connect on LinkedIn or reject
5. Edit profile page and mark profile as matched once co-founder is found
Actions: The backend actions will handle creating and updating